# Web scraping av polisens händelser

En vanlig organisation som ingår i journalisternas nyhetsnät är polisen. På polisens hemsida finns en lista över händelser i Sverige (https://polisen.se/aktuellt/polisens-nyheter/). Vi kan förvisso leta bland händelserna manuellt efter något som intresserar oss, men varför inte göra det automatiskt med hjälp av Python?

Vi kan lösa det på två sätt:

1. **Skrapa RSS-flödet**. Vi kan skrapa polisens RSS-flöde med händelser (se dme här: https://polisen.se/aktuellt/rss/). Detta är lättast eftersom [RSS](https://sv.wikipedia.org/wiki/RSS) är en standard för att utbyta information på webbplatser. Då kan vi också använda koden till alla webbplatser som har RSS-flöde. Men polisens RSS-flöde är begränsat, och alla händelser som finns på hemsidan finns inte med i RSS-flödet.
2. **Skrapa webbsidan**. Vi kan skrapa webbsidans HTML-kod. Då kan vi få ned allt som finns på hemsidan. Men om polisen ändrar sin hemsida så kan vårt program slutera fungera.

Vilken ska vi välja? Vi testar båda!


## Först, installera bibliotek

In [ ]:
# Först måste vi dock installera de bibliotek vi
# ska använda. Ändra till pip3 om pip inte fungerar.
!pip install BeautifulSoup4
!pip install lxml
!pip install requests

## 1. Skrapa RSS-flödet

Se alla RSS-flöden som polisen har: https://polisen.se/aktuellt/rss/

Vi använder detta RSS-flöde (nyheter i hela landet): https://polisen.se/aktuellt/rss/hela-landet/nyheter-hela-landet/

In [103]:
# Importera bibliotek.
from urllib import request
from lxml import html
from bs4 import BeautifulSoup
import ssl

def scrapewebpage(url):
    # Öppna URL och hämta HTML.
    UseSSL = True  # Om du får SSLError, ändra detta till False.
    if UseSSL:
        web = request.urlopen(url)
    else:
        web = request.urlopen(url, context=ssl._create_unverified_context())

    # Kontrollera att inga fel uppstod när URL öppnades.
    if web.getcode() == 200:
        return(web.read())
    else:
        print("Error %s reading %s" % str(web.getcode()), url)

# Hjälpfunktion som skrapar webbsidor och gör en soppa av allt ihop.
def makesoup(url):
    html = scrapewebpage(url)
    return(BeautifulSoup(html, "lxml-xml"))

In [104]:
# Läs in RSS-flödet och spara det i variablen rss_events.'
rss_events = makesoup("https://polisen.se/aktuellt/rss/hela-landet/handelser-i-hela-landet/")

# Du kan också testa med ett annat RSS-flöde:
#rss_events = makesoup("https://www.svd.se/feed/articles.rss")

# Ta ut alla <item> från RSS-flödet.
rss_items = rss_events.find_all("item")

In [105]:
# Hur många händelser hittade vi?
print("Hittade " + str(len(rss_items)) + " händelser.")

Hittade 200 händelser.


In [106]:
# Visa senaste 10 händelserna. Plocka bort [0:10] för att visa alla.
for item in rss_items[0:10]:
    title = item.find("title").get_text()
    print(title)

15 oktober 14.45, Trafikolycka, Malmö
15 oktober 14.21, Trafikolycka, Eskilstuna
15 oktober 13.11, Trafikolycka, personskada, Lysekil
15 oktober 13.45, Trafikolycka, Malmö
15 oktober 13.14, Rattfylleri, Borlänge
15 oktober 13.39, Trafikolycka, Östersund
15 oktober 13.35, Mord/dråp, Gävle
15 oktober 13.13, Brand, Halmstad
15 oktober 12.36, Rattfylleri, Södertälje
15 oktober 13.17, Brand, Kramfors


In [107]:
# En variabel som visar antalet sökresultat.
search_results = 0

# Visa bara trafikolyckor.
for item in rss_items:
    title = item.find("title").get_text()
    description = item.find("description").get_text()

    # Gör om texten till gemener, sök därefter efter trafikolyckor i Göteborg.
    if title.lower().find("trafikolycka") > -1 and title.lower().find("göteborg") > -1:
        search_results = search_results + 1
        print(title + ": " + description)
        print()

print("Hittade {0} resultat.".format(search_results))

15 oktober 11.48, Trafikolycka, personskada, Göteborg: En trafikolycka har inträffat i Tingstadstunneln, södergående körfält

15 oktober 08.54, Trafikolycka, personskada, Göteborg: En personbil och en fotgängare har varit involverade i en trafikolycka vid Lindholmen. 

Hittade 2 resultat.


In [108]:
# Gå igenom varje händelse och plocka ut datum, händelse och stad.
for item in rss_items:
    title = item.find("title").get_text()             # Titel på händelsen
    link = item.find("link").get_text()               # Länk till händelsen på polisens hemsida
    description = item.find("description").get_text() # Beskrivning av händelsen
    
    # Titlarna är skrivna i formatet "2017-11-07 20:40, Trafikolycka, Göteborg".
    # Då kan vi dela upp titeln i tre delar genom komma-tecknet
    # och på så vis plocka ut datum, händelse och ort.
    title_parts = title.split(",")                         # Dela sträng vid varje komma
    title_date = title_parts[0].strip()                    # Datum
    title_event = title_parts[1].strip()                   # Händelse
    title_city = title_parts[len(title_parts) - 1].strip() # Stad
    
    # Plocka ut händelser från Göteborg och Borås.
    if title_city in ["Göteborg", "Borås"]:
        print(title_date + " " + title_city + ", " + title_event + ": " + description)

15 oktober 11.48 Göteborg, Trafikolycka: En trafikolycka har inträffat i Tingstadstunneln, södergående körfält
15 oktober 08.54 Göteborg, Trafikolycka: En personbil och en fotgängare har varit involverade i en trafikolycka vid Lindholmen. 
15 oktober 07.46 Borås, Trafikolycka: En personbil och en cyklist har kolliderat vid Villastaden. 
14 oktober 15.02 Borås, Knivlagen: En kniv tas i beslag på Norrby Långgata.
14 oktober 06.53 Göteborg, Brand: Ett radhus brinner på Tanneskärsgatan.


## 2. Skrapa webbsidan

Vi kan skrapa HTML-koden från webbsidan https://polisen.se/aktuellt/polisens-nyheter/.

Här har vi dock ett problem eftersom polisens hemsida är ganska komplext uppbyggd med både 

- statisk HTML-kod (det man ser på sidan när man högerklickar och väljer "Visa källa").
- dynamisk HTML-kod (det som dyker upp på sidan genom att ett JavaScript-program hämtar information från ett [API](https://en.wikipedia.org/wiki/Application_programming_interface)). Det är detta som ändrar sidans [DOM](https://en.wikipedia.org/wiki/Document_Object_Model).

Men även om det är krångligare för oss att förstå hur sidan är uppbyggd, blir det faktiskt enklare att hämta informationen! Tack vare att polisen använder ett API som hämtar och skickar [JSON](https://en.wikipedia.org/wiki/JSON)-data blir det också enklare för oss att hämta informationen!

In [109]:
# Importera bibliotek vi behöver.
import requests
import time
from lxml import html
from bs4 import BeautifulSoup
import ssl

# Skapa funktion för att kommunicera med polisens API.
def get_police_json(pageindex=1, location=0):
    """ Anropar polisens händelser-API och returnerar JSON. 
    
    location:
        0 = Hela landet
        181 = Göteborg
    """
    url = "https://polisen.se/api/listingservice/items"
    web = requests.post(url, data = {"ContentId": "7069",
                                    "PageIndex": pageindex,
                                    "IncludeChildIndexes": False,
                                    "FilterOnSelectedLocation": (location > 0),
                                    "SelectedLocationId": location,
                                    "GeoAreaType": 1,
                                    "Filter": None, 
                                    "Latitude": 0,
                                    "Longitude":0,
                                    "ListItemType":0,
                                    "PropertyIds": []
                                   })
    json_content = web.json()
    return(json_content)

# Skapa funktion som gör det enkelt att hämta många händelser från polisen.
def get_events(maxpages=10, location=0):
    """ Hämtar händelser från polisen och returnerar lista. """
    l = []
    for pageindex in range(1, maxpages + 1):
        print("Hämtar händelser från sida {0}...".format(pageindex))
        json = get_police_json(pageindex, location)
        for event in json["List"]:
            l.append(event)
        if not json["HasMore"]:
            print("Slut på händelser.")
            break
        time.sleep(1.0)
    return(l)

In [110]:
# Hämta polisens händelser.
events = get_events(maxpages=5)

Hämtar händelser från sida 1...
Hämtar händelser från sida 2...
Hämtar händelser från sida 3...
Hämtar händelser från sida 4...
Hämtar händelser från sida 5...


In [111]:
# Hur många händelser hämtade vi?
len(events)

125

In [112]:
# Visa den senaste händelsen.
# Notera att Python börjar räkna från 0.
events[0]

{'HasImage': False,
 'HasTeaserText': False,
 'Headline': '15 oktober 14.45, Trafikolycka, Malmö',
 'ImageDescription': '',
 'ImageUrl': '',
 'IsUpdated': False,
 'ListItemType': 4,
 'Location': '',
 'MainContent': '<p>Polis och ambulans skickas till Industrigatan / Nobelv&auml;gen d&auml;r en person enligt larm ska vara p&aring;k&ouml;rd.</p>',
 'Organisation': None,
 'Preamble': 'Påkörd person, Industrigatan / Nobelvägen. ',
 'PublishedDate': '15 oktober 14.52',
 'RelatedPages': [],
 'SharedLinksData': {'SharePageByEmailUrl': 'mailto:?subject=Tips%20om%20information%20p%C3%A5%20polisens%20webbplats&body=Hej%2C%0A%0Ajag%20vill%20tipsa%20dig%20om%20en%20sida%20p%C3%A5%20polisen.se%20som%20jag%20tror%20kan%20vara%20intressant%20f%C3%B6r%20dig.%0A%0ATitta%20p%C3%A5%20den%20h%C3%A4r%20sidan%3A%0A%0Ahttps%3A%2F%2Fpolisen.se%2Faktuellt%2Fhandelser%2F2018%2Foktober%2F15%2F15-oktober-14.45-trafikolycka-malmo%2F',
  'SharePageOnFaceBookUrl': 'https://facebook.com/sharer.php?u=https%3A%2F%2Fpol

In [113]:
# Eftersom datan är sparad i en lista, kan vi använda
# en for-loop för att hämta varje händelse i listan.
for e in events:
    print(e["Headline"])

15 oktober 14.45, Trafikolycka, Malmö
15 oktober 14.21, Trafikolycka, Eskilstuna
15 oktober 13.11, Trafikolycka, personskada, Lysekil
15 oktober 13.45, Trafikolycka, Malmö
15 oktober 13.14, Rattfylleri, Borlänge
15 oktober 13.39, Trafikolycka, Östersund
15 oktober 13.35, Mord/dråp, Gävle
15 oktober 13.13, Brand, Halmstad
Polisstationen i Vivalla öppnar 16 oktober
15 oktober 12.36, Rattfylleri, Södertälje
15 oktober 13.17, Brand, Kramfors
15 oktober 12.24, Trafikolycka, Kristinehamn
15 oktober 13.06, Trafikolycka, vilt, Pajala
15 oktober 13.00, Mord/dråp, försök, Umeå
15 oktober 11.45, Stöld/inbrott, Krokom
15 oktober 11.48, Trafikolycka, personskada, Göteborg
15 oktober 12.15, Stöld/inbrott, Uppsala
15 oktober 12.16, Rattfylleri, Karlstad
15 oktober 12.39, Stöld, Enköping
15 oktober 12.07, Misshandel, Helsingborg
15 oktober 12.37, Inbrott, Karlstad
15 oktober 11.10, Trafikolycka, vilt, Skellefteå
15 oktober 12.35, Trafikolycka, Eskilstuna
15 oktober 09.30, Stöld/inbrott, Arboga
15 okto

In [114]:
# Vi kan skapa en enkel sökfunktion som
# letar igenom alla händelser vi samlat in. 
def search(word, events):
    for e in events:
        if e["Headline"].lower().find(word.lower()) > 0:
            print(e["Headline"])
            #print(l["Preamble"])
            #print()

In [115]:
# Sök igenom händelserna efter texten "Göteborg".
search("göteborg", events)

15 oktober 11.48, Trafikolycka, personskada, Göteborg
15 oktober 08.54, Trafikolycka, personskada, Göteborg


In [116]:
# Sök igenom händelserna efter texten "mord".
search("mord", events)

15 oktober 13.35, Mord/dråp, Gävle
15 oktober 13.00, Mord/dråp, försök, Umeå
14 oktober 21.58, Mord/dråp, försök, Stockholms län
15 oktober 02.48, Mord/dråp, Eskilstuna
